In [ ]:
from keras.utils import to_categorical # type: ignore
from keras_preprocessing.image import load_img
from keras.models import Sequential # type: ignore
from keras.layers import Dense, Input, Conv2D, Dropout, Flatten, MaxPooling2D # type: ignore
import os
import pandas as pd
import numpy as np


In [5]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

In [6]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels

In [7]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

angry completed
disgusted completed
fearful completed
happy completed
neutral completed
sad completed
surprised completed


In [8]:
print(train)

                                  image      label
0            images/train\angry\im0.png      angry
1            images/train\angry\im1.png      angry
2           images/train\angry\im10.png      angry
3          images/train\angry\im100.png      angry
4         images/train\angry\im1000.png      angry
...                                 ...        ...
28704  images/train\surprised\im995.png  surprised
28705  images/train\surprised\im996.png  surprised
28706  images/train\surprised\im997.png  surprised
28707  images/train\surprised\im998.png  surprised
28708  images/train\surprised\im999.png  surprised

[28709 rows x 2 columns]


In [9]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

angry completed
disgusted completed
fearful completed
happy completed
neutral completed
sad completed
surprised completed


In [10]:
print(test)
print(test['image'])

                               image      label
0          images/test\angry\im0.png      angry
1          images/test\angry\im1.png      angry
2         images/test\angry\im10.png      angry
3        images/test\angry\im100.png      angry
4        images/test\angry\im101.png      angry
...                              ...        ...
7173  images/test\surprised\im95.png  surprised
7174  images/test\surprised\im96.png  surprised
7175  images/test\surprised\im97.png  surprised
7176  images/test\surprised\im98.png  surprised
7177  images/test\surprised\im99.png  surprised

[7178 rows x 2 columns]
0            images/test\angry\im0.png
1            images/test\angry\im1.png
2           images/test\angry\im10.png
3          images/test\angry\im100.png
4          images/test\angry\im101.png
                     ...              
7173    images/test\surprised\im95.png
7174    images/test\surprised\im96.png
7175    images/test\surprised\im97.png
7176    images/test\surprised\im98.png
7177    i

In [11]:
from tqdm.notebook import tqdm

In [12]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale =  True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [13]:
train_features = extract_features(train['image']) 

  0%|          | 0/28709 [00:00<?, ?it/s]

C:\Users\shwet\AppData\Roaming\Python\Python312\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [14]:
test_features = extract_features(test['image'])

  0%|          | 0/7178 [00:00<?, ?it/s]

In [15]:
x_train = train_features/255.0
x_test = test_features/255.0

In [16]:
from sklearn.preprocessing import LabelEncoder

In [17]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [18]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [19]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [20]:
model = Sequential()
# convolutional layers
model.add(Input(shape=(48, 48, 1)))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

In [21]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [22]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 100, validation_data = (x_test,y_test)) 

Epoch 1/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 261s 1s/step - accuracy: 0.2484 - loss: 1.8351 - val_accuracy: 0.2471 - val_loss: 1.8150
Epoch 2/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 251s 1s/step - accuracy: 0.2489 - loss: 1.8146 - val_accuracy: 0.2471 - val_loss: 1.7908
Epoch 3/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 261s 1s/step - accuracy: 0.2707 - loss: 1.7597 - val_accuracy: 0.3229 - val_loss: 1.6732
Epoch 4/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 244s 1s/step - accuracy: 0.3442 - loss: 1.6339 - val_accuracy: 0.4489 - val_loss: 1.4310
Epoch 5/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 250s 1s/step - accuracy: 0.4178 - loss: 1.5031 - val_accuracy: 0.4730 - val_loss: 1.3629
Epoch 6/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 259s 1s/step - accuracy: 0.4471 - loss: 1.4336 - val_accuracy: 0.4975 - val_loss: 1.3062
Epoch 7/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 246s 1s/step - accuracy: 0.4664 - loss: 1.3853 - val_accuracy: 0.5021 - val_loss: 1.2944
Epoch 8/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 250s 1s/step - accuracy: 0.4761 - loss: 1.3566 - 

In [4]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.keras")

NameError: name 'model' is not defined